In [ ]:
from Gantry.envs.GantrySimulation import GantrySimulation
import torch
import time
import pybullet as p
import pybullet_data
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from Gantry.controller.SNS_layer import perceptor, controller_modulation, controller_closed_loop_modulation, pick_and_place, sensory_layer_2_feedback_modulation, SNS_Control_closed_loop_v1, SNS_Control_closed_loop_v2, SNS_Control_closed_loop_modulation, controller_closed_loop_v2, R, perceptor_closed_loop_modulation

## Some constant parameters used in simulation.

object_position = [0, 0, -0.34] (m)

target_position = [0.15, 0.15, -0.34] (m)

mass = 1.0 kg (Here, we use a large weight to test the effect of time history dependent control)

size scaling factor = 0.6

simulation end time = 13 (sec)

open-loop perceptor = perceptor (with an inhibitory synapse from the grasping-the-object command neuron to the lifting-the-object-up command neuron. This additional synapse can prevent the controller from transitioning to the lifting up phase too early)

closed-loop perceptor = perceptor_closed_loop_modulation (similar to the open-loop perceptor except that the force thresholds can be modulated)

open-loop controller with time history dependent control = controller_modulation (jaws use position control)

closed-loop controller with force threshold modulation control = controller_closed_loop_v2 (jaws use speed control to regulate force, time history dependent control regulates force thresholds)

force threshold gain = 0.1 (Initially multiplying force sensory signals by 10. Warning: this method may only work in simulation. Using a small gain in the experiment may lead to a force threshold lower than the noise level)

jaw pressure used in the open loop controller = 2.5 (PSI)

grasper closing speed gain = 0.3 

time constants of neurons in the perceptor = 0.05 (sec)

force modulation gain = 5 (this gain controls the strength of the modulation. The maximal force threshold = force threshold gain x force modulation gain x original force thresholds)

In [ ]:
OBJECT_POSITION = [0, 0, -0.34]
TARGET_POSITION = [0.15, 0.15, -0.34]
MASS = 1
SIZE_SCALING = 0.6

END_TIME = 13

OPEN_LOOP_PERCEPTOR = perceptor
CLOSED_LOOP_PERCEPTOR = perceptor_closed_loop_modulation
OPEN_LOOP_CONTROLLER = controller_modulation
CLOSED_LOOP_CONTROLLER = controller_closed_loop_v2
FORCE_THRESHOLD_GAIN = 0.1
PRESSURE_VALUE = 2.5
GRASPER_CLOSING_SPEED = 0.3
PERCEPTOR_TAU = 0.05
FORCE_MODULATION_GAIN = 5

## Some constant parameters used in plotting.

xlim = (1, 11)

limit for grasper command axis= (0, 0.1) (m)

limit for contact force axis = (0, 50) (N)

limit for pressure axis = (0, 3) (PSI)

In [ ]:
XLIM = (1, END_TIME - 1)
GRASPER_COMMAND_LIM = (0, 0.1)
GRASPER_COMMAND_TICKS = np.arange(0, 0.12, step=0.05)
FORCE_LIM = (0, 50)
PRESSURE_COMMAND_LIM = (0, 3)
PRESSURE_COMMAND_TICKS = np.arange(0, 3, step=1)

## Time history dependent control simulation

We the test time history dependent control on an open loop controller and a closed loop controller (v2): Without neuromodulation, the open loop controller can only send a fixed jaw positIon during grasper closing phases. Time history dependent controLler can modulate the jaw position commands so that the grasper can generate higher contact forces if the first pick attempt fails. For the closed loop controller, time history dependent controLler modulates the <s>jaw pressure commands</s> force thresholds.    

How to make the grasper fail on the first attempt: In the simulation, we use a quite heavy object (1 kg) to increase the difficulty of the task. The initial maximal jaw command of the open loop controller is set to a value lower than 0.1. This prevents the grasper from establishing strong contact with the object at the beginning. We also use a small force threshold so that the controller start picking the object up even with small contact forces. The low force threshold can be achieved by either setting a small force threshold gain for the perceptor, or using perceptor_modulation. perceptor_modulation is a special SNS that has no inhibitory synapses from the grasping-the-object command neuron to the lifting-the-object-up command neuron. Controller using this SNS tends to lift the object up earlier.

In [ ]:
OPEN_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)
OPEN_LOOP_PERCEPTOR.reset()
OPEN_LOOP_CONTROLLER.reset()
neuron_history_open_loop_modulation, sensory_history_open_loop_modulation, command_history_open_loop_modulation = pick_and_place(position_o=OBJECT_POSITION, position_t=TARGET_POSITION, perceptor=OPEN_LOOP_PERCEPTOR, controller=OPEN_LOOP_CONTROLLER, PressureValue=PRESSURE_VALUE, end_time=END_TIME, force_threshold_gain=FORCE_THRESHOLD_GAIN, zero_time_constant=True, mass=MASS, sizeScaling=SIZE_SCALING)

CLOSED_LOOP_PERCEPTOR.set_modulation_gain(FORCE_MODULATION_GAIN)
CLOSED_LOOP_PERCEPTOR.set_tau(PERCEPTOR_TAU)
CLOSED_LOOP_PERCEPTOR.reset()
CLOSED_LOOP_CONTROLLER.reset()
neuron_history_closed_loop_modulation, sensory_history_closed_loop_modulation, command_history_closed_loop_modulation = pick_and_place(position_o=OBJECT_POSITION, position_t=TARGET_POSITION, perceptor=CLOSED_LOOP_PERCEPTOR, controller=CLOSED_LOOP_CONTROLLER, end_time=END_TIME, force_threshold_gain=FORCE_THRESHOLD_GAIN, grasper_closing_speed=GRASPER_CLOSING_SPEED, zero_time_constant=True, mass=MASS, sizeScaling=SIZE_SCALING)

## Plot the time history of the contact forces, Jaw commands, and pressure commands in the above simulations.

In [ ]:
def time_history_dependent_control_plot(neuron_history_open_loop_modulation, sensory_history_open_loop_modulation, command_history_open_loop_modulation, neuron_history_closed_loop_modulation, sensory_history_closed_loop_modulation, command_history_closed_loop_modulation, xlim=XLIM, grasper_command_lim=GRASPER_COMMAND_LIM, pressure_command_lim=PRESSURE_COMMAND_LIM, force_lim=FORCE_LIM, grasper_command_ticks=GRASPER_COMMAND_TICKS, pressure_command_ticks=PRESSURE_COMMAND_TICKS):
    plt.rcParams['axes.spines.top'] = False
    plt.rcParams['axes.spines.right'] = False

    fig, ax = plt.subplots(3)
    time_history = np.arange(neuron_history_open_loop_modulation[:, 0].size) / 240
    
    ax[0].plot(time_history, command_history_open_loop_modulation[:, 3])
    ax[0].plot(time_history, command_history_closed_loop_modulation[:, 3])
    ax[0].set_ylabel('Radius')
    ax[0].set(ylim=grasper_command_lim, xlim=xlim)
    ax[0].spines['bottom'].set_visible(False)
    ax[0].axes.get_xaxis().set_visible(False)
    ax[0].axes.set_yticks(grasper_command_ticks)

    ax[1].plot(time_history, command_history_open_loop_modulation[:, 4])
    ax[1].plot(time_history, command_history_closed_loop_modulation[:, 4])
    ax[1].set_ylabel('Pressure')
    ax[1].set(ylim=pressure_command_lim, xlim=xlim)
    ax[1].spines['bottom'].set_visible(False)
    ax[1].axes.get_xaxis().set_visible(False)
    ax[1].axes.set_yticks(pressure_command_ticks)

    ax[2].plot(time_history, np.sum(sensory_history_open_loop_modulation[:, 3:5], axis=1), label='Open loop')
    ax[2].plot(time_history, np.sum(sensory_history_closed_loop_modulation[:, 3:5], axis=1), label='Closed loop')
    ax[2].set_xlabel('Time')
    ax[2].set_ylabel('Contact Force')
    ax[2].set(ylim=force_lim, xlim=xlim)
    ax[2].legend(frameon=False)

    plt.show()
    return

time_history_dependent_control_plot(neuron_history_open_loop_modulation, sensory_history_open_loop_modulation, command_history_open_loop_modulation, neuron_history_closed_loop_modulation, sensory_history_closed_loop_modulation, command_history_closed_loop_modulation)